In [10]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
import re
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, TensorBoard
from keras import metrics
import json
from helper_functions import import_data, neuralnet_model, batch_generator
from sklearn.metrics import confusion_matrix
from os import listdir

Using TensorFlow backend.


In [2]:
y_batch = y_debug
y_out = np.zeros([y_batch.shape[0], 2])
y_out[:, 0] = y_batch.reshape(-1,)
y_out[:, 1] = y_out[:, 1] + 1
y_out[:, 1] = y_out[:, 1] - y_out[:, 0]

NameError: name 'y_debug' is not defined

In [3]:
X_test, y_batch = import_data('jsons_test/' + listdir('jsons_test')[0])


y_out = np.zeros([y_batch.shape[0], 2])
y_out[:, 0] = y_batch.reshape(-1,)
y_out[:, 1] = y_out[:, 1] + 1
y_out[:, 1] = y_out[:, 1] - y_out[:, 0]
y_test = y_out

In [4]:
X_test, y_batch = import_data('jsons_test/' + listdir('jsons_test')[0])

In [5]:
autoencoder_model_created = False
model_path = 'models/basic_autoencoder1.h5'

if not autoencoder_model_created:
    model = neuralnet_model(X_test)

    batch_size = 2000
    nb_epoch = 10
    samples_per_epoch = 15

    # instantiate callbacks
    tensorboard = TensorBoard(log_dir='./autoencoder_logs', histogram_freq=2, batch_size=batch_size, write_graph=True, write_grads=True, write_images=True)
    earlystopping = EarlyStopping(monitor='val_loss', patience=2)

    # try different number of epochs - 10 gives good performanace 
    """model.fit(X_train, X_train, epochs=10, batch_size=batch_size, verbose=1,
              validation_split=0.1, callbacks = [earlystopping, tensorboard])""" # cross val to estimate test error


    model.fit_generator(generator=batch_generator('jsons_full', batch_size, samples_per_epoch),
                        epochs=nb_epoch,
                        steps_per_epoch=samples_per_epoch)


    scores = model.evaluate(X_test, y_test)
    print('Test mse = {}'.format(scores[0]))

    X_test_decoded = model.predict(X_test)
    
    model.save(model_path)

else:
    model = load_model(model_path)
    scores = model.evaluate(X_test, X_test)
    print('Test mse = {}'.format(scores[0]))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
15/15 [==============================] - 7s 481ms/step - loss: 2.1832 - mean_squared_error: 0.4277
Epoch 2/10
15/15 [==============================] - 7s 452ms/step - loss: 0.7852 - mean_squared_error: 0.2786
Epoch 3/10
15/15 [==============================] - 7s 457ms/step - loss: 0.3905 - mean_squared_error: 0.1156
Epoch 4/10
15/15 [==============================] - 7s 451ms/step - loss: 0.2576 - mean_squared_error: 0.0761
Epoch 5/10
15/15 [==============================] - 7s 444ms/step - loss: 0.2457 - mean_squared_error: 0.0704
Epoch 6/10
15/15 [==============================] - 7s 444ms/step - loss: 0.2396 - mean_squared_error: 0.0676
Epoch 7/10
15/15 [==============================] - 7s 450ms/step - loss: 0.2374 - mean_squared_error: 0.0690
Epoch 8/10
15/15 [==============================] - 7s 447ms/step - loss: 0.2272 - mean_squared_error: 0.0656


In [6]:
confusion_matrix(y_test, X_test_decoded.astype(int))

ValueError: multilabel-indicator is not supported

In [7]:
X_test_decoded

array([[9.9974149e-01, 2.5854426e-04],
       [9.9982798e-01, 1.7200824e-04],
       [9.9554551e-01, 4.4545210e-03],
       ...,
       [9.7963333e-01, 2.0366650e-02],
       [9.9997890e-01, 2.1062528e-05],
       [9.9761176e-01, 2.3882086e-03]], dtype=float32)